# Script to Consolidate Data and Create a Day Count

In [22]:
import pandas as pd
import glob

In [23]:
path = "D:/Dropbox/Data/R Shiny/20-03-20 Covid-19"

## Combine all the files together

In [24]:
all_files = glob.glob(path + "/csse_covid_19_daily_reports/*.csv")

li = []

# for files in folder, create a dataframe out of them
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    # create a date field that is generated from the file name
    df['date'] = filename.split('\\')[-1].split('.')[0]
    # add data to list
    li.append(df)

# concatinate all data (union) into final dataframe    
frame = pd.concat(li, axis=0, ignore_index=True)

# Change Country Names for consistency
frame["Country/Region"].replace({"occupied Palestinian territory": "Palestine", "Mainland China": "China", "Taiwan*": "Taiwan", "UK": "United Kingdom", "The Bahamas": "Bahamas, The", "Iran (Islamic Republic of)": "Iran", "Hong Kong SAR": "Hong Kong", "The Gambia": "Gambia, The", "Viet Nam": "Vietnam", "Korea, South": "South Korea"}, inplace=True)

In [25]:
# aggregate numbers by country
country_df = frame.groupby(['Country/Region','date']).sum().reset_index()

## Aggregate the numbers on a country level

In [26]:
country_list = []

# split the dataframe by country
for country, country_df in country_df.groupby('Country/Region'):
    # Add a day field and number it chronologically
    country_df = country_df.reset_index()
    country_df['Days'] = country_df.index + 1
    
    # Get deltas for deaths, recovered, and then new cases using pd.diff() function
    country_df['New_Deaths'] = country_df['Deaths'].diff()
    country_df['New_Recovered'] = country_df['Recovered'].diff()
    country_df['New_Confirmed'] = country_df['Confirmed'].diff()
    
    
    # add data to list
    country_list.append(country_df)
    
# concatinate all data (union) into final dataframe    
frame2 = pd.concat(country_list, axis=0, ignore_index=True)

## Cleaning the data

In [27]:
# delete newly created fields
del frame2['index']

In [28]:
# rearrange fields for easier reading
final_table = frame2[['date','Days','Country/Region','Confirmed','New_Confirmed','Deaths','New_Deaths','Recovered','New_Recovered','Latitude','Longitude']]
#replace NaN with empty string
final_table = final_table.fillna(0)
# Get active cases for the day
final_table['Active_Cases'] = final_table['Confirmed'] - final_table['Deaths'] - final_table['Recovered']

In [29]:
# for row in dataframe
for i, row in final_table.iterrows():
    # if Days == 1
    if (row['Days'] == 1):
        # C_delta == Confirmed, D_delta == Deaths, R_delta == Recovered
        final_table.at[i,'New_Confirmed'] = row['Confirmed']
        final_table.at[i,'New_Deaths'] = row['Deaths']
        final_table.at[i,'New_Recovered'] = row['Recovered']

In [30]:
final_table[final_table['Country/Region'] == 'US']

,date,Days,Country/Region,Confirmed,New_Confirmed,Deaths,New_Deaths,Recovered,New_Recovered,Latitude,Longitude,Active_Cases
3776,01-22-2020,1,US,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0
3777,01-23-2020,2,US,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0
3778,01-24-2020,3,US,2.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,2.0
3779,01-25-2020,4,US,2.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,2.0
3780,01-26-2020,5,US,5.0,3.0,0.0,0.0,0.0,0.0,0.000000,0.000000,5.0
3781,01-27-2020,6,US,5.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,5.0
3782,01-28-2020,7,US,5.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,5.0
3783,01-29-2020,8,US,5.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,5.0
3784,01-30-2020,9,US,5.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,5.0
3785,01-31-2020,10,US,6.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,6.0


In [31]:
final_table.to_csv(path + "/transformed.csv", index=False)